## 기본 합성곱 신경망 구현

In [1]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터

In [2]:
EPOCHS = 10

## 모델 정의

In [11]:
class ConvNet(tf.keras.Model):
    def __init__(self):
        super(ConvNet, self).__init__()
        conv2d = tf.keras.layers.Conv2D
        maxpool = tf.keras.layers.MaxPool2D
        self.sequence = list()
        self.sequence.append(conv2d(16, (3,3), padding='same', activation = 'relu')) # 28x28x16
        self.sequence.append(conv2d(16, (3,3), padding='same', activation = 'relu')) # 28x28x16
        self.sequence.append(maxpool((2,2))) # 14x14x16
        
        self.sequence.append(conv2d(32, (3,3), padding='same', activation = 'relu')) # 14x14x32
        self.sequence.append(conv2d(32, (3,3), padding='same', activation = 'relu')) # 14x14x32
        self.sequence.append(maxpool((2,2))) # 7x7x32
        
        self.sequence.append(conv2d(64, (3,3), padding='same', activation = 'relu')) # 7x7x64
        self.sequence.append(conv2d(64, (3,3), padding='same', activation = 'relu')) # 7x7x64
        self.sequence.append(tf.keras.layers.Flatten()) # 1568
        self.sequence.append(tf.keras.layers.Dense(2048, activation='relu'))
        self.sequence.append(tf.keras.layers.Dense(10, activation='softmax'))
        

    def call(self, x, training=False, mask=None):
        for layers in self.sequence:
            x = layers(x)
        return x
        

## 학습, 테스트 루프 정의

In [12]:
# Implement training loop
@tf.function
def train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)

    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

# Implement algorithm test
@tf.function
def test_step(model, images, labels, loss_object, test_loss, test_accuracy):
    predictions = model(images)

    t_loss = loss_object(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

## 데이터셋 준비


In [13]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
xtrain, x_test = x_train / 255.0, x_test / 255.0

# x_train : NUM_SAMPLE, 28, 28 -> (NUM_SAMPLE, 28, 28, 1)

x_train = x_train[..., tf.newaxis].astype(np.float32)
x_test = x_test[..., tf.newaxis].astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## 학습 환경 정의
### 모델 생성, 손실함수, 최적화 알고리즘, 평가지표 정의

In [14]:
# Create model
model = ConvNet()

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# Define performance metrics
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

## 학습 루프 동작

In [15]:
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(model, images, labels, loss_object, optimizer, train_loss, train_accuracy)

    for test_images, test_labels in test_ds:
        test_step(model, test_images, test_labels, loss_object, test_loss, test_accuracy)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1,
                          train_loss.result(),
                          train_accuracy.result() * 100,
                          test_loss.result(),
                          test_accuracy.result() * 100))
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

Epoch 1, Loss: 0.19837869703769684, Accuracy: 95.62833404541016, Test Loss: 2.2968344688415527, Test Accuracy: 11.350000381469727
Epoch 2, Loss: 0.061443790793418884, Accuracy: 98.125, Test Loss: 2.3996384143829346, Test Accuracy: 11.350000381469727
Epoch 3, Loss: 0.05238678678870201, Accuracy: 98.4383316040039, Test Loss: 2.4109296798706055, Test Accuracy: 11.350000381469727
Epoch 4, Loss: 0.04267407953739166, Accuracy: 98.7249984741211, Test Loss: 3.3624649047851562, Test Accuracy: 11.350000381469727
Epoch 5, Loss: 0.03963099420070648, Accuracy: 98.82333374023438, Test Loss: 5.802657127380371, Test Accuracy: 11.350000381469727
Epoch 6, Loss: 0.03513386473059654, Accuracy: 98.91500091552734, Test Loss: 9.757018089294434, Test Accuracy: 11.350000381469727
Epoch 7, Loss: 0.03367387130856514, Accuracy: 99.0199966430664, Test Loss: 17.014118194580078, Test Accuracy: 11.350000381469727
Epoch 8, Loss: 0.03324258327484131, Accuracy: 99.01000213623047, Test Loss: 33.02206802368164, Test Accur